In [11]:
# individual case output test
import os
import numpy as np
from lake_functions import *
from lake_cases import *
import datetime as dtt
import matplotlib.pyplot as plt

# PARAMETERS
#Time
nyears=2
tyear=np.tile(np.arange(1,366,1),[1,nyears]) # DOY
tyear=tyear[0];
t=np.arange(1,365*nyears+1,1) # Time in days from the first day

# Meteorological data:
meteodata_NASA =np.genfromtxt("Meteo_data_NASA.txt", usecols=tuple(np.arange(3,9,1)),skip_header=1,dtype=float)
meteodata_NASA=np.tile(meteodata_NASA,[nyears,1]) 
Ta=meteodata_NASA[:,4]  # [°C]
RH=meteodata_NASA[:,2]  # [#]
Wsp=meteodata_NASA[:,5]  # [m/s]
R=meteodata_NASA[:,1]*10**6/(24*3600)  # [W/m^2]
P=meteodata_NASA[:,3]*1000  # [Pa]

# Atmospheric variables:
rho_air=1.2  # [kg/m^3], air density
CD=0.001  # [-], drag coefficient

#Basin:
hmax=8; # [m], lake depth

# Heat fluxes:
Twinter=4
Adiff=0.066  # [-], albedo of diffuse shortwave radiation
Adir=0.1  # [-], albedo of direct shortwave radiation (approximate value for 50°N)
C=0.5  # Cloudiness
Fdir=(1-C)/((1-C)+0.5*C) 
Fdiff=0.5*C/((1-C)+0.5*C) 
Hsw0=-R*(Fdir*(1-Adir)+Fdiff*(1-Adiff))  # [W/m^2], shortwave radiation (<0) passing through the lake surface

# Gravity
g=9.81 # [m.s^(-2)]

# Water and particles properties:
Dp=1.6*10**(-6)  # [m]
rho_p=2650 # [kg/m^3]
nu=1E-6  # [m^2/s]
Vs=g*(rho_p-1000)/1000*Dp**2/(18*nu)  # [m/s], settling velocity
f_fft=10  # [%], solids content in the FFT
C_FFT=f_fft*10**4  # [g/m^3], solids concentration in the FFT

# THERMOCLINE

iceoff=(dtt.datetime(2015,4,26)-dtt.datetime(2015,1,1)).days  # iceoff DOY (based on observations)
iceon=max(t)   # initial value for the iceon DOY, it will be calculated by the simulation
h_epi=np.zeros(np.shape(t)); h_epi[0]=hmax 

# INITIALIZATION

 # Temperature:
T_epi=np.zeros(np.shape(t)); T_epi[0]=Twinter   # [°C]
T_hypo=np.zeros(np.shape(t)); T_hypo[0]=Twinter   # [°C]

 # Solids concentration:
Css_epi=np.zeros(np.shape(t)); Css_epi[0]=100   # [mg/L]
Css_hypo=np.zeros(np.shape(t)); Css_hypo[0]=100   # [mg/L]


In [12]:
#case 1
import numpy as np
import sys
from lake_functions import alphaT,rho_TCss, seasonal_therm
from statistics import mean

def case1(VAR_LAKE,Twinter=4,hmax=8):
    """
    Winter case: ice-covered lake, no heat fluxes = constant temperatures 
    and concentrations. 
     
    INPUTS:
        VAR_LAKE: dictionary containing
            T_epi, T_hypo: epilimnion and hypolimnion temperatures [°C]
            Css_epi, Css_hypo: epilimnion and hypolimnion solids concentrations [mg/L]
            h_epi: thermocline depth [m]
        Twinter: temperature of maximum density [°C]
        hmax: lake depth [m]
        
    OUTPUTS:
        VAR_LAKE: dictionary containing the same variables than VAR_LAKE but at the next time step
    """

    # -------- CODE --------
    # Temperature
    T_epi1 = Twinter
    T_hypo1 = Twinter
    # solid concentrations
    Css_epi1 = 100
    Css_hypo1 = 100

    h_epi1 = hmax
    
    VAR_LAKE={"T_epi":T_epi1,"T_hypo":T_hypo1,"Css_epi":Css_epi1,"Css_hypo":Css_hypo1,"h_epi":h_epi1}

    return VAR_LAKE
    

In [15]:
def case2(VAR_LAKE,Hsurf,Hsw0,tyear,iceon,Vs,C_FFT,hmax=8,g=9.81,Cpw=4200,Twinter=4,A0=7.8*10**6):
    """
    Fall turnover case: one box with heat fluxes.
     
    INPUTS:
        VAR_LAKE: dictionary containing
            T_epi, T_hypo: epilimnion and hypolimnion temperatures [°C]
            Css_epi, Css_hypo: epilimnion and hypolimnion solids concentrations [mg/L]
            h_epi: thermocline depth [m]
        Hsurf: surface heat flux [W/m^2]
        Hsw0: surface radiation flux [W/m^2]
        tyear: time as DOY
        iceon: iceon date as DOY
        Vs: settling velocity [m/s]
        C_FFT: solids concentration in the FFT [g/m^3]
        hmax: lake depth [m]
        g: gravitational acceleration [m/s^2]
        Cpw: heat capacity of water [J.kg^(-1).K^(-1)]
        Twinter: temperature of maximum density [°C]
        A0: lake surface area [m^2]
        
        
    OUTPUTS:
        VAR_LAKE: dictionary containing the same variables than VAR_LAKE but at the next time step
        iceon: iceon date as DOY (modified only if ice formation after initial iceon date)
    """
    # ----Initial setup----
    h_epi2 = hmax  # Thermocline depth
    T2 = VAR_LAKE["T_epi"]  # Temperature, constant
    T_hypo = VAR_LAKE["T_hypo"] # Hypolimnion temperature for stable condition
    Css2 = VAR_LAKE["Css_epi"] # Initial solid concentration constants
    
    # ----Special process in fall----
    # --Redistribution--
    alpha2 = alphaT(T2)
    kd = 0.325 + 66 * Css2 / 1000  # mg/L
    B = alpha2*g*(Hsurf + Hsw0*(1-np.exp((0.325+66*Css2)*hmax))) / (rho_w0 * Cpw)

    # --stability and concentration calculate--
    if B > 0: # unstable, fully mixed, new epi. depth is constant
        hepi_new = hmax
# ABOUT this part, I'm not sure if I understood the comments and process correctly. I just wrote this as a temporary version to see if they work.
        # Hsw0, consider different layer # ALSO, A BIT CONFUSED WITH THIS COMMENT
        Qnet2 = -(Hsurf + Hsw0)*A0
        dT2 = (Qnet2/(m_w0*Cpw))/24*60*60
        Tepi_new = T2 + dT2
        Thypo_new = Tepi_new
        
        if Tepi_new <= 4: #additional safety check
            if t[k] > 200:
                iceon = t[k]
            else:
                print("Warning, Ice in Summer!")
                Tepi_new = Twinter
                Thypo_new = Twinter
        else: # no ice
            # calculate velocities
            rho_30 = rho_TCss(T2, 30)
            N_fft2 = g*(rho_30-rho_w0)/(0.5*rho_fft0) 
            Vr = (1+2*0.2)*B/(N_fft2*hmax)
            # water mass
            m_w0 = rho_w0*A0*hmax 
            # new concentration
            C_sed = -Css2*Vs*A0*24*60*60/A0*hmax
            C_red = C_FFT*Vr*A0*24*60*60/A0*hmax
            Cssnew2 = Css2 + C_sed + C_red
            Csse_new = Cssnew2
            Cssh_new = Cssnew2
            # safety check
            if Cssnew2 < 0:
                print("Warning, Solid Concentration Error!")
            elif Vr < 0:
                print("Warning, Resettling Velocity Error!")
    else: # stable
        T_avg = T2 + Hsurf *24*3600/(Cpw*rho_w0*h_epi)
        T_surf = 2*T_avg - T_hypo
        # density of 2 layers
        rho1 = rho_TCss(T_surf,Css)
        rho2 = rho_TCss(T_hypo,Css)
        rho_avg = (rho1+rho2)/2
        # N^2
        N2 = g*(rho1 - rho2)/ rho_avg / h_epi
        # new depth of thermocline
        hepi_new = seasonal_therm(h_epi,3600*24,B,N2)
        # ---Identify: stratified---
        if hepi_new > (hmax - 0.1): # stratified ignored
            hepi_new = hmax
            Tepi_new = T_avg
            Thypo_new = T_avg
        else:
            T_therm = T_surf - (hepi_new/hmax)*(T_surf - T_hypo)
            Tepi_new = 0.5 * (T_therm + T_surf)
            Thypo_new = 0.5 * (T_hypo + T_therm)
            
        Csse_new = Css2
        Cssh_new = Css2
    
    # iceon
    if Tepi_new <= Twinter and iceon == max(t):
        iceon = tyear

    VAR_LAKE={"T_epi":Tepi_new,"T_hypo":Thypo_new,"Css_epi":Csse_new,"Css_hypo":Cssh_new,"h_epi":hepi_new}
    return VAR_LAKE, iceon

In [18]:
# main code test
for k in np.arange(300): 
    VAR_LAKE={"T_epi":T_epi[k],"T_hypo":T_hypo[k],"Css_epi":Css_epi[k],"Css_hypo":Css_hypo[k],"h_epi":h_epi[k]}
    
    if k < iceoff or k > iceon:
        VAR_LAKE = case1(VAR_LAKE)
        
    elif h_epi[k] >= hmax - 0.11:
        VAR_LAKE, iceon = case2(VAR_LAKE)
        
    else:
        VAR_LAKE = case1(VAR_LAKE)
    lake_list = []
    lake_list.append(VAR_LAKE.copy())
    print(k, lake_list)
    print(iceon)

0 [{'T_epi': 4, 'T_hypo': 4, 'Css_epi': 100, 'Css_hypo': 100, 'h_epi': 8}]
730
1 [{'T_epi': 4, 'T_hypo': 4, 'Css_epi': 100, 'Css_hypo': 100, 'h_epi': 8}]
730
2 [{'T_epi': 4, 'T_hypo': 4, 'Css_epi': 100, 'Css_hypo': 100, 'h_epi': 8}]
730
3 [{'T_epi': 4, 'T_hypo': 4, 'Css_epi': 100, 'Css_hypo': 100, 'h_epi': 8}]
730
4 [{'T_epi': 4, 'T_hypo': 4, 'Css_epi': 100, 'Css_hypo': 100, 'h_epi': 8}]
730
5 [{'T_epi': 4, 'T_hypo': 4, 'Css_epi': 100, 'Css_hypo': 100, 'h_epi': 8}]
730
6 [{'T_epi': 4, 'T_hypo': 4, 'Css_epi': 100, 'Css_hypo': 100, 'h_epi': 8}]
730
7 [{'T_epi': 4, 'T_hypo': 4, 'Css_epi': 100, 'Css_hypo': 100, 'h_epi': 8}]
730
8 [{'T_epi': 4, 'T_hypo': 4, 'Css_epi': 100, 'Css_hypo': 100, 'h_epi': 8}]
730
9 [{'T_epi': 4, 'T_hypo': 4, 'Css_epi': 100, 'Css_hypo': 100, 'h_epi': 8}]
730
10 [{'T_epi': 4, 'T_hypo': 4, 'Css_epi': 100, 'Css_hypo': 100, 'h_epi': 8}]
730
11 [{'T_epi': 4, 'T_hypo': 4, 'Css_epi': 100, 'Css_hypo': 100, 'h_epi': 8}]
730
12 [{'T_epi': 4, 'T_hypo': 4, 'Css_epi': 100, 'Css